In [6]:
import pandas as pd
import numpy as np

In [7]:
excluded_companies = [
    "BOEING CO/THE", "INTEL CORP", "MCDONALD'S CORP", "APPLE INC", "AMAZON.COM INC",
    "MICROSOFT CORP", "CHEVRON CORP", "EXXON MOBIL CORP", "LOCKHEED MARTIN CORP"
]

# Function to calculate percentage unrealized loss
def calculate_loss_percentage(df):
    df['Base Unrealized Gain/Loss'] = df['Base Unrealized Gain/Loss'].replace(',', '', regex=True).astype(float)
    df['Base Market Value'] = df['Base Market Value'].replace(',', '', regex=True).astype(float)
    
    # Calculate the percentage of unrealized loss
    df['Percentage Unrealized Loss'] = df.apply(
        lambda row: (row['Base Unrealized Gain/Loss'] / row['Base Market Value']) * 100 if row['Base Market Value'] != 0 else 0,
        axis=1
    )

    israel_st_loss = df[df['Security Description1'].str.contains("ISRAEL", na=False)]['Percentage Unrealized Loss']
    other_loss = df[~df['Security Description1'].str.contains("ISRAEL", na=False)]['Percentage Unrealized Loss']
    
    return israel_st_loss.describe(), other_loss.describe()

# Function to filter out specified companies and recalculate losses
def filter_and_recalculate_losses(df):
    # Filter out the specified companies
    filtered_df = df[~df['Security Description1'].str.contains('|'.join(excluded_companies), case=False, na=False)]
    
    # Recalculate percentage of unrealized loss
    filtered_israel_st_loss, filtered_other_loss = calculate_loss_percentage(filtered_df)
    
    return filtered_israel_st_loss, filtered_other_loss

# List of file paths and corresponding months
file_paths = [
    ('../data/input/2024/BNY Mellon GASB Holding Daily 29 Feb 2024 - Redacted.csv', 'Feb 2024'),
    ('../data/input/2024/Copy of BNY Mellon GASB Holdings Daily_31 Oct 2023.csv', 'Oct 2023'),
    ('../data/input/2024/Copy of BNY Mellon GASB Holdings Daily_30 Nov 2023.csv', 'Nov 2023'),
    ('../data/input/2024/Copy of BNY Mellon GASB Holdings Daily_30 Dec 2023.csv', 'Dec 2023'),
    ('../data/input/2024/Copy of BNY Mellon GASB Holdings Daily_29 Sep 2023.csv', 'Sep 2023'),
    ('../data/input/2024/BNY Mellon GASB Holdings Daily 31 Jan 2024 - Redacted.csv', 'Jan 2024'),
    ('../data/input/2024/BNY Mellon GASB Holding Daily 31 Mar 2024 - Redacted.csv', 'Mar 2024')
]

# Initialize lists to store results
monthly_results = []

# Process each file
for file_path, month in file_paths:
    gasb_data = pd.read_csv(file_path)
    filtered_gasb_israel_st_loss, filtered_gasb_other_loss = filter_and_recalculate_losses(gasb_data)
    
    monthly_results.append((month, filtered_gasb_israel_st_loss, filtered_gasb_other_loss))

# Combine results into a summary table for each month
summary_tables = []
for month, israel_st, other in monthly_results:
    summary_table_israel = pd.DataFrame(israel_st).reset_index().rename(columns={'index': 'Statistic', 'Percentage Unrealized Loss': 'Value'})
    summary_table_israel['Investment Type'] = 'ISRAEL ST'
    summary_table_israel['Month'] = month
    
    summary_table_other = pd.DataFrame(other).reset_index().rename(columns={'index': 'Statistic', 'Percentage Unrealized Loss': 'Value'})
    summary_table_other['Investment Type'] = 'Other Investments'
    summary_table_other['Month'] = month
    
    summary_table = pd.concat([summary_table_israel, summary_table_other])
    summary_tables.append(summary_table)

# Combine all monthly tables into one dataframe
monthly_summary_df = pd.concat(summary_tables).reset_index(drop=True)

# Calculate average for all months
average_israel_st_loss = pd.DataFrame([res[1] for res in monthly_results]).mean().reset_index().rename(columns={0: 'Value'})
average_israel_st_loss['Statistic'] = average_israel_st_loss['index']
average_israel_st_loss['Investment Type'] = 'ISRAEL ST'
average_israel_st_loss['Month'] = 'Average'

average_other_loss = pd.DataFrame([res[2] for res in monthly_results]).mean().reset_index().rename(columns={0: 'Value'})
average_other_loss['Statistic'] = average_other_loss['index']
average_other_loss['Investment Type'] = 'Other Investments'
average_other_loss['Month'] = 'Average'

# Combine average results into a summary table
average_summary_table = pd.concat([average_israel_st_loss, average_other_loss], ignore_index=True)

# Combine monthly and average summary tables
final_summary_df = pd.concat([monthly_summary_df, average_summary_table], ignore_index=True)

final_summary_df


/var/folders/4r/jhc99db14v58mjmncgtry0kc0000gn/T/ipykernel_20462/1424752929.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Base Unrealized Gain/Loss'] = df['Base Unrealized Gain/Loss'].replace(',', '', regex=True).astype(float)
/var/folders/4r/jhc99db14v58mjmncgtry0kc0000gn/T/ipykernel_20462/1424752929.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Base Market Value'] = df['Base Market Value'].replace(',', '', regex=True).astype(float)
/var/folders/4r/jhc99db14v58mjmncgtry0kc0000gn/T/ipyke

,Statistic,Value,Investment Type,Month,index
0,count,5.000000,ISRAEL ST,Feb 2024,NaN
1,mean,-1.024458,ISRAEL ST,Feb 2024,NaN
2,std,0.980623,ISRAEL ST,Feb 2024,NaN
3,min,-2.187842,ISRAEL ST,Feb 2024,NaN
4,25%,-1.498112,ISRAEL ST,Feb 2024,NaN
...,...,...,...,...,...
123,min,-27.659061,Other Investments,Average,min
124,25%,-7.086172,Other Investments,Average,25%
125,50%,-2.186503,Other Investments,Average,50%
126,75%,0.807125,Other Investments,Average,75%


### Summary of GASB Holdings Unrealized Loss Percentages

#### Key Findings:
1. **ISRAEL ST Investments:**
   - **Mean Unrealized Loss:** The mean percentage unrealized loss for ISRAEL ST investments across the analyzed months is approximately -1.72%.
   - **Standard Deviation:** The standard deviation is about 1.17%, indicating moderate variability around the mean.
   - **Monthly Loss Range:** The minimum monthly unrealized loss recorded is approximately -2.89%, and the maximum is around -0.29%.

2. **Other Investments:**
   - **Mean Unrealized Loss:** The mean percentage unrealized loss for other investments across the analyzed months is approximately 3.83%.
   - **Standard Deviation:** The standard deviation is significantly higher at around 21.54%, indicating high variability in the unrealized losses.
   - **Monthly Loss Range:** The monthly unrealized losses ranged widely, with a minimum of about -27.66% and a maximum of around 91.18%.

#### Interpretation:
- **Consistency in ISRAEL ST Losses:** The unrealized losses for ISRAEL ST investments remain relatively consistent over the months, with losses ranging from -2.89% to -0.29%. This suggests a stable but negative performance trend for ISRAEL ST investments.
